## 1. 데이터 로드

In [2]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc9_0_32 = pd.read_csv("../binutils_gcc3~9_op0~4_csv/"+'gcc9'+"_0_32.csv", index_col=0)

# 형태 출력
print(gcc9_0_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc9_0_32.reset_index(inplace=True, drop=True)
print('reset_index 완료')
print('input data shape')
gcc9_0_32.head()

(2668228, 2)
reset_index 완료
input data shape


,hex,bin
0,55,1
1,48,0
2,89,0
3,e5,0
4,48,0


## 2. Data 확인

In [3]:
# DATA 모양 확인
gcc9_0_32.values
print(gcc9_0_32.shape)

# 함수의 갯수
gcc9_0_32['bin'].value_counts()

# 함수 평균길이
gcc9_0_32['bin'].value_counts()[0]/gcc9_0_32['bin'].value_counts()[1]

array([['55', 1],
       ['48', 0],
       ['89', 0],
       ...,
       ['f8', 0],
       ['c9', 0],
       ['c3', 0]], dtype=object)

(2668228, 2)


0    2663795
1       4433
Name: bin, dtype: int64

600.901195578615

## 3. 함수의 갯수, 바이트 갯수의 빈공간 np.array 생성 

In [8]:
x = np.full((gcc9_0_32['bin'].value_counts()[1],45000), None, dtype='object')
x.shape
x.dtype

(4433, 45000)

dtype('O')

## 4. 함수 빈공간에 함수별로 한줄씩 값 대입

In [9]:
data = gcc9_0_32.values

row = -1
col = 0
col_list=[]

# data => [55,1], [89,1], [e5,1] ....
for i in data:
    # i[0] = 55, 89, e5... /  i[1] = 1, 0, 0, ....
    if i[1]==1:
        col_list.append(col) # col_list의 각 함수 길이들 넣는다.
        
        row+=1
        col=0
        x[row][col] = i[0]
#        print(row,col, end='/') # 오류났을 때 풀어보면 몇번째 행에서 값을 초과하는지 알 수 있음.
    else:
        x[row][col+1] = i[0]
        #if row >=1385:
        #    print(row,col, end=' ')
        col+=1
    
print('대입 완료')    

대입 완료


In [10]:
max(col_list)

44112

## 5. 최적의 x 찾기

In [11]:
# 예시
x[:, :5]
print(len(col_list),'\n')

# 최소 3gram ~ 7gram 까지확인
for j in range(3, 7):
    tuple_ls = []
    for k in x[:, :j]:
        tuple_ls.append(tuple(k))

    set_ls = set(tuple_ls)
    print(j, 'gram일 때 ', len(set_ls))

array([['55', '48', '89', 'e5', '48'],
       ['55', '48', '89', 'e5', '48'],
       ['55', '48', '89', 'e5', '48'],
       ...,
       ['55', '48', '89', 'e5', '48'],
       ['55', '48', '89', 'e5', '48'],
       ['55', '48', '89', 'e5', '48']], dtype=object)

4433 

3 gram일 때  1
4 gram일 때  1
5 gram일 때  16
6 gram일 때  34


In [12]:
# 임의 테스트
tls = []
for t in x[:, :4]:
    tls.append(tuple(t))
    
stls = set(tls)
stls

{('55', '48', '89', 'e5')}